# Data Wrangling and Cleaning

## Importing necessary libraries

In [ ]:
import glob
import pandas as pd

## Load CSV files into a pandas dataframe

Create a function that takes a path to a CSV file as anargument and returns a pandas dataframe. This function loops through the CSV files in the path and creates a pandas dataframe for each file with the dataframe name as the CSV file name.

In [ ]:
def convert_csv_to_df(csv_directory_path):
    dfs = {}
    print('-' * 150)
    print('Now converting csv files to dataframes...')
    print('-' * 150)
    for csv_file in glob.glob(csv_directory_path + '/*.csv'):
        print(f'Working on {csv_file} ...')
        print('Converting to dataframe...')
        df = pd.read_csv(csv_file)
        print('Done... Now Getting filename...')
        filename = csv_file.split('/')[-1].replace('.csv', '')
        print('Done... Renaming dataframe...')
        exec(f"dfs['df_{filename}'] = df")
    print('-' * 150)
    print('Done converting csv files to dataframes...')
    return dfs

Running the function:

In [ ]:
dfs = convert_csv_to_df('data-analytics-files')

Sorting the dictionary of dataframes based on key names

In [ ]:
dfs.keys()
sorted_keys = sorted(dfs.keys())
dfs_sorted = {key: dfs[key] for key in sorted_keys}
dfs_sorted.keys()

## Data Cleaning

Create a function that takes in a dataframe as an argument. The function counts the null values in each column and then finds the columns where all the values are null. These columns are then removed from the dataframe. A second check for null values is performed to see if there are any columns where all the values are null in the dataframe.

In [ ]:
def count_and_delete_nulls(df):
    num_rows = df.shape[0]
    print(f'This dataframe contains {num_rows} rows')
    print('Counting nulls in dataframe')
    null_count = pd.isna(df).sum()
    print(null_count)
    all_nan_cols = df.columns[df.isna().all()]
    print(all_nan_cols)
    print('Dropping nulls columns in dataframe')
    df.drop(columns=all_nan_cols, axis=1, inplace=True)
    print('Checking for any other columns with all null values')
    print(df.columns[df.isna().all()])
    print('Done')
    return df

Running the function and returning the resulting dataframes into a list in order to concatenate them into a single dataframe later.

In [ ]:
dfs_list = []
for df_name, df_value in dfs_sorted.items():
    print('-' * 150)
    print(f'Working on {df_name}')
    print('-' * 150)
    df = count_and_delete_nulls(df_value)
    dfs_list.append(df)

Below, we are counting the number of rows in each dataframe and the number of nulls in each column. Following this we are counting the number of nulls in total then filling these values with 0.

In [ ]:
clean_dfs_list = []
for df in dfs_list:
    num_rows = df.shape[0]
    print(f'This dataframe contains {num_rows} rows')
    null_count = df.isna().sum()
    if null_count.any():
        print(f'This dataframe contains null values in some columns')
        null_count_all = df.isna().sum().sum()
        print(f' In total, this dataframe contains {null_count_all} null values')
    else:
        print(f'This dataframe does not contain null values in any of the columns')
    print(f'Filling these null values in the dataframe with 0s')
    df.fillna(0, inplace=True)
    clean_dfs_list.append(df)
    

In this block of code we are exploring the columns remaining in each of the dataframes. Some dataframes have the same columns but some have additional columns.

In [ ]:
for df in clean_dfs_list:
    print('Checking the columns of the dataframe:')
    columns = df.columns.to_list()
    print(columns)
    columns_total = len(columns)
    print(f'In total there are {columns_total} columns in the dataframe.')

## Data Integration

Now, we are integratng the dataframes into one merged dataframe. We then count the number of rows and check if the resulting dataframe contains null values as some columns didn't exist in the original individual dataframes. These null values are then filled with 0s. Following this we export the resulting dataframe to a csv file.

In [ ]:
print('Concatenating the dataframes together...')
df = pd.concat(clean_dfs_list).reset_index(drop=True)
num_rows = df.shape[0]
print(f'Number of rows: {num_rows}')
print('Counting the null values in the combined dataframe...')
for col in df.columns:
    null_count = df[col].isnull().sum()
    print(f'{col}: {null_count} null values')
print('Filling the null values in the combined dataframe with 0s...')
df.fillna(0, inplace=True)

# print('Saving the combined dataframe...')
# df.to_csv('combined_data.csv', index=False)



Next we explore the resulting dataframe

In [22]:
import numpy as np

df.columns
df.info()
df['ArrDelay'].info()
value = 'SAN'
position = np.where(df.values == value)
print(position)
df.head().to_csv('data.csv', index=False)
df['ArrDelay'].head()
df.tail().to_csv('data2.csv', index=False)
